# Code generation

Tomado de [Whole-stage code generation: Spark 2.0's Tungsten engine](https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/6122906529858466/293651311471490/5382278320999420/latest.html)

In [1]:
from time import perf_counter

## Spark < 2.0

In [2]:
spark.conf.set("spark.sql.codegen.wholeStage", False)

In [3]:
res = spark.range(1000 * 1000 * 1000).join(spark.range(1000), "id").selectExpr("count(*)").explain()

== Physical Plan ==
HashAggregate(keys=[], functions=[count(1)])
+- HashAggregate(keys=[], functions=[partial_count(1)])
   +- Project
      +- BroadcastHashJoin [id#0L], [id#2L], Inner, BuildRight, false
         :- Range (0, 1000000000, step=1, splits=1)
         +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [id=#22]
            +- Range (0, 1000, step=1, splits=1)




## Spark >= 2.0

In [4]:
spark.conf.set("spark.sql.codegen.wholeStage", True)

In [5]:
spark.range(1000 * 1000 * 1000).join(spark.range(1000), "id").selectExpr("count(*)").explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('count(1), Some(org.apache.spark.sql.Column$$Lambda$1571/0x0000000840cb3040@2bca2061))]
+- Project [id#10L]
   +- Join Inner, (id#10L = id#12L)
      :- Range (0, 1000000000, step=1, splits=Some(1))
      +- Range (0, 1000, step=1, splits=Some(1))

== Analyzed Logical Plan ==
count(1): bigint
Aggregate [count(1) AS count(1)#16L]
+- Project [id#10L]
   +- Join Inner, (id#10L = id#12L)
      :- Range (0, 1000000000, step=1, splits=Some(1))
      +- Range (0, 1000, step=1, splits=Some(1))

== Optimized Logical Plan ==
Aggregate [count(1) AS count(1)#16L]
+- Project
   +- Join Inner, (id#10L = id#12L)
      :- Range (0, 1000000000, step=1, splits=Some(1))
      +- Range (0, 1000, step=1, splits=Some(1))

== Physical Plan ==
*(2) HashAggregate(keys=[], functions=[count(1)], output=[count(1)#16L])
+- *(2) HashAggregate(keys=[], functions=[partial_count(1)], output=[count#19L])
   +- *(2) Project
      +- *(2) BroadcastHashJoin [id#10L], [id

## Tiempos

In [6]:
spark.conf.set("spark.sql.codegen.wholeStage", True)

In [7]:
p1 = perf_counter()
res = spark.range(1000 * 1000 * 1000).join(spark.range(1000), "id").selectExpr("count(*)").collect()
p2 = perf_counter()
print(res[0])

Row(count(1)=1000)


In [8]:
p2 - p1

5.184639289975166